In [1]:
import praw
import pandas as pd
import json
import urllib.request

### Using Praw to Pull Submissions

In [2]:
reddit = praw.Reddit(client_id='iAajnINW1rc_9Q', client_secret='-oxEXPNDghVqscnTykypNn34gGrFsA', 
                     user_agent='Garlic_Negative')

In [55]:
posts = []
hot_posts = reddit.subreddit('AmItheAsshole').top(limit=500)
for post in hot_posts:
    posts.append([post.title,post.score, post.id, post.num_comments, post.selftext,post.link_flair_text])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'num_comments', 'body','flair'])

In [56]:
posts

,title,score,id,num_comments,body,flair
0,META: This sub is moving towards a value syste...,80921,d6xoro,6249,I’ve enjoyed reading and posting on this sub f...,META
1,"UPDATE, AITA for despising my mentally handica...",72783,azvko1,1995,"I'm back like I said I would be,. My [original...",UPDATE
2,AITA For suing my girlfriend after she had my ...,70808,gr8bp3,2787,I'll try to keep this short. I had a [1967 Imp...,Not the A-hole
3,UPDATE: AITA for wanting to go to the funeral ...,67578,cjetsa,2,I want to sincerely thank everyone who comment...,Update
4,AITA for pretending to get fired when customer...,63536,e5k3z2,3659,I am a high schooler with a weekend job at a c...,Not the A-hole
...,...,...,...,...,...,...
495,AITA for telling my girlfriend that her phobia...,23537,ea4hnu,2895,My (20) girlfriend (21) has a very severe phob...,Not the A-hole
496,"META At any point, the advice you're reading c...",23506,b2x90i,764,"This seems like a really weird meta post, but ...",META
497,UPDATE: AITA for not agreeing to a paternity t...,23475,j9ueq4,1598,This is an update to my other [post](https://w...,UPDATE
498,AITA For Telling My Wife That If She Quits Her...,23461,cyj5v1,3777,My wife wants to quit her job. She wants to be...,Not the A-hole


In [47]:
posts.to_csv('top_500_posts.csv')

### Using Praw to Pull Comments

In [35]:
comments = []
for item in ids:
    submission = reddit.submission(id=item)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments.append([comment.id,item, comment.score, comment.body])
comments = pd.DataFrame(comments,columns=['id','submission_id', 'score', 'body'])

In [36]:
comments

,id,submission_id,score,body
0,gt2oml3,mhylc3,61,I gotta say... In the months I've been on this...
1,gt1rppj,mhylc3,55,"This subreddit for the next week: \n\n""AITA fo..."
2,gtc6inn,mhylc3,40,I really wish we could get a cull on posts tha...
3,gt2906u,mhylc3,28,"So many posts here are actually asking ""are th..."
4,gt5vgj4,mhylc3,33,Can we have an assholes of the month thread? A...
...,...,...,...,...
10096,gton0ov,mhylc3,14,Anyone can feel insecure and ashamed of their ...
10097,gtrf55u,mhylc3,3,Yes! Big-thigh girl right here.
10098,gtooeg5,mhylc3,-1,Now you’re just making scenarios up to fit you...
10099,gtop1eb,mhylc3,10,There is tho: OP and the other guy in the room...


### Using Pushshift.io API to grab older submissions

In [66]:
#Starting pull from January 1st 2014
utc = 1388552400
url = f'https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&size=100&num_comments=>500&after={utc}'
data = urllib.request.urlopen(url).read()
obj = json.loads(data)
df2 = pd.DataFrame(obj['data'])
df2 = df2[['title', 'score', 'id', 'num_comments','selftext','created_utc']]
df3 = df2.copy()

In [67]:
#Run this loop until it stops, sometimes it stops early because of a 502 
# error so run it a few times to be sure you get all of the data
#For some reason there's no data available from 2014 to 2019

#Can only pull 100 at a time so creating loop
for i in range(1,499):
    print(i)
    url = f'https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&size=100&num_comments=>500&after={utc}'
    data = urllib.request.urlopen(url).read()
    obj = json.loads(data)
    df2 = pd.DataFrame(obj['data'])
    df2 = df2[['title', 'score', 'id', 'num_comments','selftext','created_utc']]
    df3 = pd.concat([df3, df2])
    utc = df3['created_utc'].max()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


HTTPError: HTTP Error 502: Bad Gateway

In [73]:
#title, score, id, num_comments, num_comments, body
#df3 = df2[['title', 'score', 'id', 'num_comments','selftext','created_utc']]
df3[df3['score']!=1]

,title,score,id,num_comments,selftext,created_utc
0,AITA for not believing my daughter is non-binary?,2068,9aeeed,905,Recently my daughter has 'come out' to me as n...,1535276025
1,AITA for being upset my boyfriend didn’t get m...,287,9eyfox,693,He has no money until he’s paid at the end of ...,1536678944
2,AITA for not wanting to date a transgender per...,2766,9gix1f,1385,Let me just say up front that I have nothing a...,1537177551
3,AITA for reporting my classmates for doing weed?,1427,9ihcqq,1667,"About a year ago, I became quite religious. An...",1537790836
4,AITA for flipping out at dinner about my frien...,2717,9k4ef9,779,I have a friend that is a nurse. While we were...,1538291368
...,...,...,...,...,...,...
26,AITA for telling my co-worker to stop being su...,2,fkuaog,1659,Throwaway cause I don't want this associated w...,1584555584
27,AITA for telling my childless sister to stop p...,4,fkvcva,648,My sister lived in the family home until recen...,1584559254
28,AITA for telling a friend she needs to stop ha...,10,fkz8xv,987,We're a group of 20 somethings who met through...,1584572901
29,AITA for purposefully having a 10-11 year enga...,5,fl3fwk,524,My fiancée and I have been together since we w...,1584589966


In [74]:
df3.reset_index(inplace=True)
df3.drop(columns='index',inplace=True)
df3

,title,score,id,num_comments,selftext,created_utc
0,AITA for not believing my daughter is non-binary?,2068,9aeeed,905,Recently my daughter has 'come out' to me as n...,1535276025
1,AITA for being upset my boyfriend didn’t get m...,287,9eyfox,693,He has no money until he’s paid at the end of ...,1536678944
2,AITA for not wanting to date a transgender per...,2766,9gix1f,1385,Let me just say up front that I have nothing a...,1537177551
3,AITA for reporting my classmates for doing weed?,1427,9ihcqq,1667,"About a year ago, I became quite religious. An...",1537790836
4,AITA for flipping out at dinner about my frien...,2717,9k4ef9,779,I have a friend that is a nurse. While we were...,1538291368
...,...,...,...,...,...,...
5295,AITA for literally not wanting to be pregnant ...,1,fp1561,520,So I (39F) have been married to my husband Mik...,1585180464
5296,AITA for ignoring my sisters request to stop c...,1,fp2i9w,1166,My husband and I live in a two bedroom apartme...,1585185628
5297,AITA for attempting to sell my boyfriend’s toys?,1,fp41ug,529,Throwaway because my boyfriend knows my real r...,1585191860
5298,"AITA for not wanting to ""tone down"" my wedding?",1,fp4kaj,1652,My fiance and I wanted to do something small f...,1585194042


In [107]:
dff = df3.copy()
dff = dff[~dff['flair'].isin(['IMPORTANT','Probably Fake', 'weeabo h8r', 
                              'probably fake', 'Probably fake', 'Happy New Year!', 'UPDATE', 'META',
                              'Fake', 'Fake - removed', 'Judgement denied.', 'Shitpost', 'Not enough info', 
                              'Party Pies', 'Quality Shitpost', 'Community Discussion', 'NEWS',
                             'I crave validation!','Fake story/Everyone Sucks', 'Fake Story','Update'
                             'Judgment denied.',None])]

In [108]:
dff['flair'].unique()

array(['asshole', 'not the a-hole', 'justified asshole', 'everyone sucks',
       'Not the A-hole', 'Asshole', 'Everyone Sucks',
       'Record Setting Asshole', 'Not the A-hole (oof)', 'META Asshole',
       'No A-holes here', 'Update', 'Judgment denied.'], dtype=object)

In [76]:
### Need to grab flair tags with praw
ids = df3['id'].values
flairs  = []
for item in ids: 
    submission = reddit.submission(id=item)
    flairs.append(submission.link_flair_text)
df3['flair']=flairs

In [112]:
dff

,title,score,id,num_comments,selftext,created_utc,flair
1,AITA for being upset my boyfriend didn’t get m...,287,9eyfox,693,He has no money until he’s paid at the end of ...,1536678944,asshole
2,AITA for not wanting to date a transgender per...,2766,9gix1f,1385,Let me just say up front that I have nothing a...,1537177551,not the a-hole
3,AITA for reporting my classmates for doing weed?,1427,9ihcqq,1667,"About a year ago, I became quite religious. An...",1537790836,asshole
4,AITA for flipping out at dinner about my frien...,2717,9k4ef9,779,I have a friend that is a nurse. While we were...,1538291368,justified asshole
5,AITA For having drunk sex with a girl who didn...,1,9l0vf9,643,I met this girl a couple of weeks back in a pu...,1538571142,asshole
...,...,...,...,...,...,...,...
5290,AITA for not wanting my wife’s birthday present?,1,foh9yl,740,"The other week, my wife broke my gaming system...",1585098410,Not the A-hole
5291,AITA for how I treated a waitress,1,foo3cr,1252,I was flying home from a work trip and my flig...,1585132428,Not the A-hole
5293,AITA for no longer giving a struggling neighbo...,1,fotbrh,1413,When my (30F) husband and I got our first hous...,1585153981,Not the A-hole
5296,AITA for ignoring my sisters request to stop c...,1,fp2i9w,1166,My husband and I live in a two bedroom apartme...,1585185628,Not the A-hole


In [51]:
dff.to_csv('over_10K_posts.csv')